<a href="https://colab.research.google.com/github/NiclasFenton-Wiegleb/schlager-lyrics-bot/blob/main/7b_Schlager_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Training 7b model*

In [ ]:
#Install dependencies

!pip install "transformers==4.34.0" "datasets==2.13.0" "peft==0.5.0" "accelerate==0.23.0" "bitsandbytes==0.41.1" "trl==0.4.7" "safetensors>=0.3.1" "huggingface_hub" --upgrade

In [2]:
#Login to huggingface

from huggingface_hub import notebook_login

notebook_login()

In [3]:
#Import Libraries

from datasets import load_dataset
from random import randrange
import pandas as pd
from datasets.dataset_dict import DatasetDict, Dataset
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from transformers import TrainingArguments
from trl import SFTTrainer
import os
from peft import AutoPeftModelForCausalLM

In [4]:
#Import data

df_lyrics = pd.read_csv('schlager_songs_v2.csv')
df_lyrics['lyrics'][10]

'{"status"false,"reason""Unexpected error occurred (no quota cost) Please try again later"}'

In [10]:
#Clean and prepare data to be turned into dataset

context=[]

instruction=[]

for ind in df_lyrics["lyrics"].index:
  lyrics = df_lyrics["lyrics"].iloc[ind]
  verse_1 = df_lyrics["verse_1"].iloc[ind]
  if type(verse_1) != str:
    continue
  if lyrics == '{"status"false,"reason""Unexpected error occurred (no quota cost) Please try again later"}':
    continue
  else:
    try:
      #Instruction
      instruction.append(verse_1)

      #Context
      context.append(lyrics)
    except:
      continue

df_dataset = pd.DataFrame(columns=['instruction', 'context'])

df_dataset['instruction'] = instruction
df_dataset['context'] = context

df_dataset.head(15)

,instruction,context
0,Hey Hey Hey (Hey) In einer Kneipe bei Bier un...,Hey\n Hey\n Hey\n (Hey) In einer Kneipe bei B...
1,"Jetzt geht es los, los, los","Jetzt geht es los, los, los\n Die Krüge hoch,..."
2,"Du bist die schönste, schönste, schönste Frau...","Du bist die schönste, schönste, schönste Frau..."
3,Wie jedes Jahr Malle wir sind wieder da Ole O...,Wie jedes Jahr\n Malle wir sind wieder da\n O...
4,Der Wind is kalt und rau,Der Wind is kalt und rau\n Und mer wisse janz...
5,Oh oh oh yeoh Oh oh oh yeoh Oh oh oh yeoh Oh ...,Oh oh oh yeoh\n Oh oh oh yeoh\n Oh oh oh yeoh...
6,Ich hab Muskeln und 'nen durchtrainierten Bauch,Ich hab Muskeln und 'nen durchtrainierten Bau...
7,Wir sind wieder da Wie jedes Jahr Hier ist An...,Wir sind wieder da\n Wie jedes Jahr\n Hier is...
8,"Oh Baby, komm doch her zu mir","Oh Baby, komm doch her zu mir\n Es ist mir eg..."
9,Du bist der geilste Ort der Welt,Du bist der geilste Ort der Welt\n Bist unser...


In [11]:
#Turn dataframe into dataset

dataset_lyrics = Dataset.from_pandas(df_dataset)
dataset_lyrics

Dataset({
    features: ['instruction', 'context'],
    num_rows: 1046
})

In [12]:
#Check sample from dataset

dataset_lyrics[randrange(len(dataset_lyrics))]

{'instruction': ' Liebe Gemeinde - Liebe Partygemeinde Es ist an der Zeit ein Lied zu singen Was auf der ganzen Welt verstanden wird',
 'context': " Liebe Gemeinde -\n Liebe Partygemeinde\n Es ist an der Zeit ein Lied zu singen\n Was auf der ganzen Welt verstanden wird\n Und dieses Lied heißt\n Laudato si, o mio signore\n Laudato si, o mio signore\n Laudato si, o mio signore (auf geht's, Gemeinde! Die Hände in den Himmel)\n Laudato si, o mio signore\n Laudato si, o mio signore\n Laudato si, o mio signore\n Laudato si, o mio signore\n Laudato si, o mio signore\n Sei gepriesen für Felder und für Wiesen\n Sei gepriesen für Laminat und Fliesen\n Sei gepriesen für zwei und drei mal niesen\n Sei gepriesen, denn du bist wunderbar, yeah\n Laudato si, o mio signore\n Laudato si, o mio signore\n Laudato si, o mio signore\n Laudato si, o mio signore\n Laudato si, o mio signore\n Laudato si, o mio signore\n Laudato si, o mio signore\n Laudato si, o mio signore\n Sei gepriesen für Urlaub und Safari

In [13]:
#Creating formatting function with prompt

def format_instruction(sample):
	return f"""### Instruction:
Benuzte den gegebenen Input um ein Schlager Lied zu schreiben.

### Input:
{sample['instruction']}

### Response:
{sample['context']}
"""

In [14]:
#Test formatting function

print(format_instruction(dataset_lyrics[randrange(len(dataset_lyrics))]))

### Instruction:
Benuzte den gegebenen Input um ein Schlager Lied zu schreiben.

### Input:
 Sommersonnenschein, der in dein Zimmer fällt

### Response:
 Sommersonnenschein, der in dein Zimmer fällt
 Hell und warm dich umarmt
 Und der Schattenblick, der deine Tage trübt
 Wird auf einmal wieder klar
 Feuer wird aus Eis
 Wenn dich ihr Zauber leise berührt
 Und ein grauer Tag
 Strahlt mit einem Mal
 Und du hörst die Melodie
 Voll Gefühl und voll Poesie
 Wer Liebe lebt
 Wird unsterblich sein
 Wer Liebe lebt
 Ist niemals allein
 Such sie an jedem neuen Tag
 Wer Liebe lebt
 Wird unsterblich sein
 Wer Liebe lebt
 Mal lächelt sie dich an
 Und wird ein Leben lang nah dir steh'n
 Mmh, mal liegt das ganze Glück
 In einem Augenblick
 Und du suchst die Melodie
 Voll Gefühl und voll Poesie
 Wer Liebe lebt
 Wird unsterblich sein
 Wer Liebe lebt
 Ist niemals allein
 Such sie an jedem neuen Tag
 Neuen Tag




In [15]:
#Set whether to use flash attention (Ampere GPU only)

use_flash_attention = False

In [16]:
#Load base model and tokenizer

# Hugging Face model id (German fin-tuned Llama instance)
model_id = "LeoLM/leo-hessianai-7b"

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    use_cache=False,
    use_flash_attention_2=use_flash_attention,
    device_map="auto",
)
model.config.pretraining_tp = 1


tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [17]:
# Prep Lora layer to be trained

# LoRA config based on QLoRA paper
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=64,
        bias="none",
        task_type="CAUSAL_LM",
)

# prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [18]:
# Set training hyperparameters

args = TrainingArguments(
    output_dir="./content/drive/MyDrive/AiCore/trained_model/schlager-bot-002",
    num_train_epochs=3,
    per_device_train_batch_size=6 if use_flash_attention else 4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=5e-4,
    # bf16=True, # Only on A100
    fp16=True, # On V100
    # tf32=True, #Only on A100
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    disable_tqdm=True # disable tqdm since with packing values are in correct
)

In [20]:
#Set trainer settings

max_seq_length = 500 # max sequence length for model and packing of the dataset

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_lyrics,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=format_instruction,
    args=args,
)

In [21]:
# Maximising the space available for model loading on GPU
os.environ['PYTORCH_CUDA_ALLOC_CONF']="max_split_size_mb:512"

In [22]:
# train
trainer.train() # there will not be a progress bar since tqdm is disabled

# save model
trainer.save_model()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 1.561, 'learning_rate': 0.0005, 'epoch': 0.08}
{'loss': 1.3671, 'learning_rate': 0.0005, 'epoch': 0.15}
{'loss': 1.4412, 'learning_rate': 0.0005, 'epoch': 0.23}
{'loss': 1.3799, 'learning_rate': 0.0005, 'epoch': 0.31}
{'loss': 1.3417, 'learning_rate': 0.0005, 'epoch': 0.38}
{'loss': 1.2687, 'learning_rate': 0.0005, 'epoch': 0.46}
{'loss': 1.2544, 'learning_rate': 0.0005, 'epoch': 0.53}
{'loss': 1.2845, 'learning_rate': 0.0005, 'epoch': 0.61}
{'loss': 1.2931, 'learning_rate': 0.0005, 'epoch': 0.69}
{'loss': 1.2702, 'learning_rate': 0.0005, 'epoch': 0.76}
{'loss': 1.2999, 'learning_rate': 0.0005, 'epoch': 0.84}
{'loss': 1.3235, 'learning_rate': 0.0005, 'epoch': 0.92}
{'loss': 1.2809, 'learning_rate': 0.0005, 'epoch': 0.99}
{'loss': 1.274, 'learning_rate': 0.0005, 'epoch': 1.07}


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 1.2067, 'learning_rate': 0.0005, 'epoch': 1.03}
{'loss': 1.3404, 'learning_rate': 0.0005, 'epoch': 1.1}
{'loss': 1.1896, 'learning_rate': 0.0005, 'epoch': 1.18}
{'loss': 1.1983, 'learning_rate': 0.0005, 'epoch': 1.26}
{'loss': 1.1596, 'learning_rate': 0.0005, 'epoch': 1.33}
{'loss': 1.2062, 'learning_rate': 0.0005, 'epoch': 1.41}
{'loss': 1.1192, 'learning_rate': 0.0005, 'epoch': 1.48}
{'loss': 1.243, 'learning_rate': 0.0005, 'epoch': 1.56}
{'loss': 1.1762, 'learning_rate': 0.0005, 'epoch': 1.64}
{'loss': 1.1613, 'learning_rate': 0.0005, 'epoch': 1.71}
{'loss': 1.1883, 'learning_rate': 0.0005, 'epoch': 1.79}
{'loss': 1.2005, 'learning_rate': 0.0005, 'epoch': 1.87}
{'loss': 1.1471, 'learning_rate': 0.0005, 'epoch': 1.94}
{'loss': 1.1519, 'learning_rate': 0.0005, 'epoch': 2.02}
{'loss': 1.1386, 'learning_rate': 0.0005, 'epoch': 2.1}


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 1.09, 'learning_rate': 0.0005, 'epoch': 2.05}
{'loss': 1.0673, 'learning_rate': 0.0005, 'epoch': 2.13}
{'loss': 1.0432, 'learning_rate': 0.0005, 'epoch': 2.21}
{'loss': 1.11, 'learning_rate': 0.0005, 'epoch': 2.28}
{'loss': 1.0129, 'learning_rate': 0.0005, 'epoch': 2.36}
{'loss': 0.9119, 'learning_rate': 0.0005, 'epoch': 2.44}
{'loss': 1.0244, 'learning_rate': 0.0005, 'epoch': 2.51}
{'loss': 0.9583, 'learning_rate': 0.0005, 'epoch': 2.59}
{'loss': 1.0409, 'learning_rate': 0.0005, 'epoch': 2.66}
{'loss': 1.0284, 'learning_rate': 0.0005, 'epoch': 2.74}
{'train_runtime': 1582.7577, 'train_samples_per_second': 1.983, 'train_steps_per_second': 0.248, 'train_loss': 1.1972497607612125, 'epoch': 2.76}


In [24]:
# Push peft adapter to huggingface hub

peft_model_id = "niclasfw/schlager-bot-002"
trainer.push_to_hub(peft_model_id) #, use_auth_token=True)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/134M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

'https://huggingface.co/niclasfw/schlager-bot-002/tree/main/'

*Merge Model and Adapter and Push to Hub*

In [25]:
# Load base model and tokenizer

model_id = "LeoLM/leo-hessianai-7b"

# load base LLM model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [26]:
#Load adapter layer, merge model and push to huggingface hub

peft_model_id = "niclasfw/schlager-bot-002"

#Accelerate Model with Text Generation Inference
model = AutoPeftModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=peft_model_id,
    low_cpu_mem_usage=True,
)

# Merge LoRA and base model
merged_model = model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")

# push merged model to the hub
merged_model.push_to_hub(peft_model_id)
tokenizer.push_to_hub(peft_model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using pad_token, but it is not set yet.
Using pad_token, but it is not set yet.


pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.89G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/7.18G [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.88G [00:00<?, ?B/s]

Using pad_token, but it is not set yet.
Using pad_token, but it is not set yet.


CommitInfo(commit_url='https://huggingface.co/niclasfw/schlager-bot-002/commit/c1be7c76a0eaa11040e2b9c20a24b6b1848735ca', commit_message='Upload tokenizer', commit_description='', oid='c1be7c76a0eaa11040e2b9c20a24b6b1848735ca', pr_url=None, pr_revision=None, pr_num=None)

*7b Model Inference Testing*

In [27]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

In [28]:
model_id = "niclasfw/schlager-bot-002"

# load base LLM model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [29]:
device = "cuda"

In [30]:
#Maximising the space available for model loading on GPU
os.environ['PYTORCH_CUDA_ALLOC_CONF']="max_split_size_mb:512"

In [34]:
# model = model.to(device)
sample = f"""Ich weiss ein Geheimniss!\n
Dieter Bohlen hat keine Haare, sondern ein Tupee!\n
Wir trinken auf Dieter Bohlens Tupee\n
"""

prompt = f"""### Instruction:
Benuzte den gegebenen Input um ein Schlager Lied zu schreiben.

### Input:
{sample}

### Response:
"""

for n in range(5):
        input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
        # with torch.inference_mode():
        outputs = model.generate(input_ids=input_ids, pad_token_id=tokenizer.eos_token_id, max_new_tokens=500, do_sample=True, top_p=0.9,temperature=0.9)

        print(f"Version {n+1}")
        print(f"Prompt:\n{sample}\n")
        print(f"Generated output:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")

Version 1
Prompt:
Ich weiss ein Geheimniss!

Dieter Bohlen hat keine Haare, sondern ein Tupee!

Wir trinken auf Dieter Bohlens Tupee



Generated output:
 Dieter Bohlen hat keine Haare, sondern ein Tupee!
 Wir trinken auf Dieter Bohlens Tupee
 Wir trinken auf Dieter Bohlens Tupee
 Dieter Bohlen hat keine Haare, sondern ein Tupee!
 Wir trinken auf Dieter Bohlens Tupee
 Dieter Bohlen hat keine Haare, sondern ein Tupee!
 Wir trinken auf Dieter Bohlens Tupee
 Dieter Bohlen hat keine Haare, sondern ein Tupee!
 Wir trinken auf Dieter Bohlens Tupee
 Dieter Bohlen hat keine Haare, sondern ein Tupee!
 Wir trinken auf Dieter Bohlens Tupee


Version 2
Prompt:
Ich weiss ein Geheimniss!

Dieter Bohlen hat keine Haare, sondern ein Tupee!

Wir trinken auf Dieter Bohlens Tupee



Generated output:
 Dieter Bohlen hat keine Haare, sondern ein Tupee!
 Wir trinken auf Dieter Bohlens Tupee
 Und auf seine beiden Augenbraunen!
 Wir trinken auf Dieter Bohlen, weil er der geilste Mensch ist
 Wir singen, wir si